<a href="https://colab.research.google.com/github/vodnalashiva131/INFO-5731/blob/main/vodnala_shiva_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Wednesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [ ]:
import requests
import csv

stored_proc = "data science"
attributes = "title,year,abstract"


web_url = f"http://api.semanticscholar.org/graph/v1/paper/search/bulk?query={stored_proc}&fields={attributes}"
url = requests.get(web_url).json()

if 'total' in url:
    print(f"Will retrieve up to 10,000 documents out of an estimated {r['total']} documents")
else:
    print("Retrieving process of the documents is failed")
    exit()

retrived_documents = 0
retriveing_limits = 10000

with open("retrived_papers.csv", "w", newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Title", "Year", "Abstract"])

    while retrived_documents < retriveing_limits:
        if "data" in url:
            for paper in url["data"]:
                writer.writerow([paper.get("title", "N/A"), paper.get("year", "N/A"), paper.get("abstract", "N/A")])
                retrived_documents += 1
                if retrived_documents >= retriveing_limits:
                    break
            print(f"Retrieved {retrived_documents} papers...")
        else:
            print("Error while retrieving data")
            break

        if "token" in r and retrived_documents < retriveing_limits:
            r = requests.get(f"{url}&token={r['token']}").json()
        else:
            break

print(f"Done! Retrieved {retrived_documents} papers total.")

Will retrieve up to 10,000 documents out of an estimated 654257 documents
Retrieved 1000 papers...
Retrieved 2000 papers...
Retrieved 3000 papers...
Retrieved 4000 papers...
Retrieved 4999 papers...
Retrieved 5998 papers...
Retrieved 6998 papers...
Retrieved 7998 papers...
Retrieved 8998 papers...
Retrieved 9998 papers...
Retrieved 10000 papers...
Done! Retrieved 10000 papers total.


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

df = pd.read_csv("retrived_papers.csv")

def clean_text(text):
    if pd.isnull(text):
        return ""

    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)

    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word.lower() not in stop_words])

    text = text.lower()

    return text

df['Clean_Abstract'] = df['Abstract'].apply(clean_text)

porter_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

def stem_and_lemmatize(text):
    try:
        text = ' '.join([porter_stemmer.stem(word) for word in text.split()])
        text = ' '.join([wordnet_lemmatizer.lemmatize(word) for word in text.split()])
    except Exception as e:
        text = ''
    return text

df['Clean_Abstract'] = df['Clean_Abstract'].apply(stem_and_lemmatize)

df.to_csv("cleaned_cleared_papers.csv", index=False)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
import pandas as pd
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
import spacy

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

nlp = spacy.load("en_core_web_sm")

clean_text = df['Clean_Abstract'].dropna().tolist()

parts_of_speech = []
named_entitity_recognition = []
dependent_outputs = []

for sentence in clean_text:
    tokens = word_tokenize(sentence)

    parts_of_speech_tags = pos_tag(tokens)
    parts_of_speech.extend(parts_of_speech_tags)

    chunked = ne_chunk(parts_of_speech_tags)
    named_entitity_recognition.extend(chunked)

    doc = nlp(sentence)
    dependent_outputs.extend([(token.text, token.dep_, token.head.text) for token in doc])

In [ ]:
pos_df = pd.DataFrame(parts_of_speech, columns=['Word', 'POS_Tag'])
dependency_df = pd.DataFrame(dependent_outputs, columns=['Word', 'Dependency', 'Head'])

print("POS", pos_df['POS_TAGS'].value_counts())

pos_df.to_csv("total_pos_results.csv", index=False)
dependency_df.to_csv("dependencies_results.csv", index=False)

POS Counts: NN      1073592
JJ       322449
VBP       72782
NNS       46660
IN        32279
RB        27210
VB        25230
VBD       24762
FW        16811
VBZ       13623
CD        12399
NNP        9638
VBN        8993
MD         7171
JJR        5059
JJS        4222
RBR        1739
CC         1564
DT          866
VBG         731
RP          324
WP          233
RBS         230
PRP         210
WP$         200
WRB         142
TO           95
PRP$         51
WDT          44
SYM          26
POS          23
EX           11
UH           10
NNPS          5
PDT           3
''            2
Name: POS_Tag, dtype: int64


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [ ]:
"""The given task was very good but a little difficult. i was facing difficulty while gathering and cleansing heavy components of data.almost 10,000 research papers abstracts from
semantic scholar and cleaning the texts data by removing stopwords,numeric's, and also noise in the data and conducting stemming and lemmatization.In this kind of assignments
i got a chance to work on NLP wwhich helpmed to learn new concepts in these assignment. this approaches included name entity identification, constituency parsing ,
POS tagging and other dependency parsing.this methods are used to extract relevant information.this assignment is done by dividing the assignment in to smaller jobs and done the work.
i have used many various libraries such as numpy,spacy,pandas for retriving data.and process them. this assignment helped me to gain skills that are i am not expertise of."""
